## Data Pre-Processing
This notebook allows a simple pre-processing of the data, necessary for behavioral analysis. The steps enable the computation of a day-time frame, save the mice's genotype in the database, set the Water point position and return the total distance travelled.

In [3]:
%matplotlib inline

import sys
sys.path.insert(1, "../")


import pandas as pd
import numpy as np
import time
from datetime import *
import sqlite3
from lmtanalysis.FileUtil import getFilesToProcess
import matplotlib.pyplot as plt
import colorsys
from collections import Counter
import seaborn as sns

from lmtanalysis.Animal import *
from lmtanalysis.Measure import *
import matplotlib.pyplot as plt
from lmtanalysis.Event import *
from lmtanalysis.Util import *
from scripts.InteractionMatrix import *
from lmtanalysis import BuildEventTrain3, BuildEventTrain4, BuildEventFollowZone, BuildEventRear5, BuildEventFloorSniffing,\
    BuildEventSocialApproach, BuildEventSocialEscape, BuildEventApproachContact,\
    BuildEventApproachRear, BuildEventGroup2, BuildEventGroup3, BuildEventGroup4,\
    BuildEventStop, BuildEventWaterPoint

from tkinter.filedialog import askopenfilename
from lmtanalysis.EventTimeLineCache import EventTimeLineCached

Ask file to process

In [4]:
print( "Select file name in window" )
files = getFilesToProcess()

Select file name in window


## Manual Night Input Parameters
To configure nights manually, you will need to input the corresponding frame numbers. 
The first cell provides the beginning and end of the experiment. The second cell displays the different nights (dark phase) beginning in numbers of frame. These have to be feed in the 3rd code. 

In [3]:
#Display starting and ending time of the experiment

for file in files:
    connection = sqlite3.connect( file )
    start = getStartInDatetime( file )
    print ( "Starting time", start )
    
    end = getEndInDatetime(file)
    print ( "Ending Time", end )
        

Starting time 2020-06-06 11:45:06.058000
Ending Time 2020-06-08 10:54:26.324000


In [4]:
    #MANUAL INPUT : Make sure to put the correct day of experiment start in the 'then' parameter
    then = datetime.datetime(2020, 6, 6, 19, 0, 0)  
    duration = then - start                        
    duration_in_s = int(duration.total_seconds())
    duration_in_frames = duration_in_s*30
    night_1st_end = duration_in_frames + 1296000
    night_2nd_start = night_1st_end + 1296000
    night_2nd_end = night_2nd_start + 1296000
    night_3nd_start = night_2nd_end + 1296000
    print ( "1st Dark phase end frame", duration_in_frames)
    print ( "2nd Dark phase start frame", night_1st_end )
    print ( "2nd Dark phase end frame", night_2nd_start)
    print ( "3rd Dark phase start frame", night_2nd_end )
    print ( "End of 3rd dark phase", night_3nd_start )

1st Dark phase end frame 782790
2nd Dark phase start frame 2078790
2nd Dark phase end frame 3374790
3rd Dark phase start frame 4670790
End of 3rd dark phase 5966790


## Manual Night Input
The previous values are now used to save the dark phases of the experiment into the database.

In [5]:
#Manual Night Input, enter the values computed above

from lmtanalysis.Util import convert_to_d_h_m_s, getFrameInput

class FileProcessException(Exception):
    pass

   
def process( file ):
    
    connection = sqlite3.connect( file )     

    print( "--------------")
    print( "Current file: ", file )
    print( "Loading existing events...")
    nightTimeLine = EventTimeLine( connection, "night" , None, None, None , None )        
    
    
    print( "--------------")    
    print( "Event list:" )
    for event in nightTimeLine.eventList:
        print( event )
    print( "--------------")
    
    answer = input("remove existing nights [Y/N] ? :")
    if answer.lower() == "y":
        print("Removing existing nights")
        nightTimeLine.eventList.clear();
        nightTimeLine.endRebuildEventTimeLine(connection, deleteExistingEvent = True )
    else:
        print("Keeping existing nights.")
    
    nightTimeLine.eventList.clear();
    
    while True:
        print( "Manually add night event:")
        startFrame = getFrameInput("Start Frame (return to stop)")
        if startFrame==None:
            return
        
        endFrame = getFrameInput("End Frame (return to stop)")
        if endFrame==None:
            return
        
        nightTimeLine.addEvent( Event( int(startFrame), int(endFrame) ) )
        nightTimeLine.endRebuildEventTimeLine(connection, deleteExistingEvent = True )



print("Code launched.")

files = getFilesToProcess()

chronoFullBatch = Chronometer("Full batch" )    

if ( files != None ):

    for file in files:
        try:
            print ( "Processing file" , file )
            process( file )
        except FileProcessException:
            print ( "STOP PROCESSING FILE " + file , file=sys.stderr  )

print( "*** ALL JOBS DONE ***")
        

Code launched.
Processing file D:/live mouse tracker data/Experiment 20/Experiment 20.sqlite
--------------
Current file:  D:/live mouse tracker data/Experiment 20/Experiment 20.sqlite
Loading existing events...
night  Id( None , None , None , None ) Min/maxFrame: ( None / None ) Loaded ( 2  records loaded in  84.85777568817139 S )
--------------
Event list:
Event	start:	782816	end:	2078818
Event	start:	3374790	end:	4670785
--------------
remove existing nights [Y/N] ? :N
Keeping existing nights.
Manually add night event:
Start Frame (return to stop) : 
*** ALL JOBS DONE ***


In [6]:
# Set Genotype
#WARNING ! The genotypes have to be entered in the format "KO" for Knock-out and "WT" for control
for file in files:
    
    print("---------------")
    print("Current file: " , file )
    
    connection = sqlite3.connect( file )
    
    pool = AnimalPool( )
    pool.loadAnimals( connection )
    
    for animal in pool.getAnimalList():
        print( "****")
        
        genotype = input( "Type new genotype for RFID "+ str(animal.RFID)+ " Current Genotype: " + str(animal.genotype) + " ([Enter] to keep existing one):")
        genotype = genotype.strip()
        if len(genotype) > 0:
            print("setting ", genotype , " to " , animal.RFID )
            animal.setGenotype( genotype )
        else:
            print( "keeping genotype", animal.genotype )
        
    print("Genotype saved in database.")

---------------
Current file:  D:/live mouse tracker data/Experiment 18/Experiment 18.sqlite
Loading animals.
Fields available in lmtanalysis:  ['ID', 'RFID', 'GENOTYPE', 'NAME']
SQL Query: SELECT ID,RFID,NAME,GENOTYPE FROM ANIMAL ORDER BY GENOTYPE
Animal Id:1 Name:A RFID:000022986511 Genotype:None User1:None
Animal Id:2 Name:B RFID:RFID_B Genotype:None User1:None
****
Type new genotype for RFID 000022986511 Current Genotype: None ([Enter] to keep existing one):KO
setting  KO  to  000022986511
****
Type new genotype for RFID RFID_B Current Genotype: None ([Enter] to keep existing one):KO
setting  KO  to  RFID_B
Genotype saved in database.


## Water point
The following command defines the coordinates of the water bottle. 
This position can be modified in the python file "BuildEventWaterPoint.py" in the 'scripts' folder. 

In [9]:
#Set the position of the water bottle

%run ./BuildEventWaterPoint.py

## Total distance travelled
As a pre-processing of the data, the following command displays the total distance traveled during the time defined, in the surface defined. These parameters can be modified in the code below. 

In [5]:
#Prompt the distance travelled

for file in files:
    # connect to database
    connection = sqlite3.connect( file )
    # create an animalPool, which basically contains your animals
    animalPool = AnimalPool()
    # load infos about the animals
    animalPool.loadAnimals( connection )       
    # load all detection (positions) of all animals for defined time
    # accepted formats are : number of frames, oneMinute, oneDay
    animalPool.loadDetection( start = 0, end = 2*oneDay )
    # filter detection by area (in cm from the top left of the cage, x1, y1, x2, y2)
    #animalPool.filterDetectionByArea( 20, 20, 30, 30 );
        
    for animal in animalPool.getAnimalList():
            
            # print RFID of animal
            print ( "Animal : " , animal.RFID )
            # distance traveled by animal (in cm):            
            print( "Distance traveled in area: (in centimeter): " , animal.getDistance( ) )

Loading animals.
Fields available in lmtanalysis:  ['ID', 'RFID', 'GENOTYPE', 'NAME']
SQL Query: SELECT ID,RFID,NAME,GENOTYPE FROM ANIMAL ORDER BY GENOTYPE
Animal Id:2 Name:B RFID:000022986392 Genotype:KO User1:None
Animal Id:3 Name:C RFID:000022986491 Genotype:KO User1:None
Animal Id:1 Name:A RFID:000022986387 Genotype:WT User1:None
Animal Id:4 Name:D RFID:000022986562 Genotype:WT User1:None
Animal Id:2 Name:B RFID:000022986392 Genotype:KO User1:None : Loading detection.
SELECT FRAMENUMBER, MASS_X, MASS_Y, MASS_Z, FRONT_X, FRONT_Y, FRONT_Z, BACK_X, BACK_Y, BACK_Z,REARING,LOOK_UP,LOOK_DOWN FROM DETECTION WHERE ANIMALID=2 AND FRAMENUMBER>=0 AND FRAMENUMBER<=5184000
Animal Id:2 Name:B RFID:000022986392 Genotype:KO User1:None   3315827  detections loaded in 773.1812393665314 seconds.
Animal Id:3 Name:C RFID:000022986491 Genotype:KO User1:None : Loading detection.
SELECT FRAMENUMBER, MASS_X, MASS_Y, MASS_Z, FRONT_X, FRONT_Y, FRONT_Z, BACK_X, BACK_Y, BACK_Z,REARING,LOOK_UP,LOOK_DOWN FROM DE